# Harmonic Coordinates for Character Articulation

In [199]:
import igl
import numpy as np
import meshplot as mp
import ipywidgets as iw
import scipy.sparse as sp
import triangle as tr
import matplotlib.pyplot as plt

## Construct Cage
Munually add the cage points one by one interactively.

In [200]:
# load object
v, f = igl.read_triangle_mesh('data/woody-hi.off')
# normalization
v -= v.min(axis=0)
v /= v.max()

In [201]:
p = mp.plot(v,f,shading={"width":300,"height":300})
p.save("imgs/woody-hi.html")

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

Plot saved to file imgs/woody-hi.html.


In [202]:
# load marker
with np.load('data/octa_sphere_5.npz') as npl:
    v_s, f_s = npl['v'], npl['f']

In [203]:
# construct the cage
cage = []
        
add_button = iw.Button(description="Add Point")
show_button = iw.Button(description="Show Cage")

# Set Callback
def add_clicked(b):
    point=sf.coord[1:]
    cage.append(point)
    paint_ui.add_points(np.array([point]),shading={"point_size":0.1})
    
def show_clicked(b):
    cage_tmp=np.array(cage)
    if len(cage)>1:
        for i in range(1,len(cage)):
            paint_ui.add_lines(cage_tmp[i-1], cage_tmp[i])
        paint_ui.add_lines(cage_tmp[0], cage_tmp[-1])
    
add_button.on_click(add_clicked)
show_button.on_click(show_clicked)

# Display Buttons
display(iw.HBox([add_button, show_button]))

# Meshplot
paint_ui = mp.plot(v,f,shading={"wireframe": True, "width":400, "height":400})
paint_ui.add_mesh(v_s*0.1, f_s,shading={"flat":False},c=np.array([1,0,0]))
def sf(radius,x,y,z):
    paint_ui.update_object(oid = 1, vertices = v_s*radius + np.array([x,y,z]))
    sf.coord = [radius,x,y,z]
mp.interact(sf, 
            radius = iw.FloatSlider(min=0.01, max=1, value=0.01),
            x = iw.FloatSlider(min=-0.1, max=1.2, value=0, step=0.01),
            y = iw.FloatSlider(min=-0.1, max=1.2, value=0, step=0.01),
            z = iw.FloatSlider(min=-0.1, max=1.2, value=0, step=0.01))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

interactive(children=(FloatSlider(value=0.01, description='radius', max=1.0, min=0.01), FloatSlider(value=0.0,…

<function __main__.sf>

In [175]:
cage = np.array(cage)

In [176]:
save = False
if save:
    paint_ui.save("imgs/cage.html")
    np.save('data/woody-hi-cage.npy', cage)

## Harmonic Coordinates

### Triangulate the cage
Use the library `triangle` to generate mesh inside the cage for 2D. It first adds random points inside the cage then apply Delaunay to triangulate the points.

\[TODO\] For 3D, need PyMesh. To avoid error, please create 2D cage in the previous step.

In [204]:
# load cage
cage=np.load('data/woody-hi-cage.npy')

In [178]:
# triangulate
def triangulate(cage):
    cage_edge = np.array([[i,i+1] for i in range(cage.shape[0])])
    cage_edge[-1, -1] = 0

    cage_dict = dict(vertices=cage[:,:2], segments=cage_edge, segment_markers=np.ones(cage.shape[0]))
    tri = tr.triangulate(cage_dict, 'pqa0.005')

    tri_v = tri['vertices']  # the first #cage.shape[0] vertices are cage vertices
    tri_f = tri['triangles']
    tri_v = np.column_stack((tri_v, np.zeros((tri_v.shape[0],1))))  # 2d -> 3d

    return cage_edge, tri_v, tri_f

In [179]:
cage_edge, tri_v, tri_f = triangulate(cage)

In [180]:
# plot the triangles
p = mp.plot(tri_v, tri_f, shading={"wireframe": True,"width":300,"height":300}, c=np.array([0,1,1]))
p.add_mesh(v,f)
p.add_points(cage,shading={"point_size":0.15})
p.save("imgs/triangulation.html")

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4250000…

Plot saved to file imgs/triangulation.html.


### Compute harmonic weight for triangle vertices
Need to solve Laplacian <=> Lx=0 s.t. boundary constraints. Each vertice after triangulation has a weight, corresponding to original cage vertices.
#### (1) Compute the triangle boundary conditions
The triangle boundary means the vertices after triangulation that lie on the original cage edges. We already know the coordinate of the cage vertices, which is 1 at the dimension of its corresponding index and 0 at other dimensions. So in this step, we compute the coordinates of these augmented cage vertices. The coordinate of the point that lies on the cage edge (vi, vj) is supposed to have positive values between 0 and 1 at i-th and j-th dimension, and to be 0 at other dimensions. 

- First, get boundary vertices.
- Next, compute boundary conditions. For a triangle boundary vertice, loop all cage edges to see which edge it lies on. A point vi lies on the line (v1, v2) if the dist(vi, v1) + dist(vi, v2) = dist(v1, v2). Because the sum of the length of two edges is greater than the other in any triangles.

In [181]:
# get triangle boundary vertices
def get_bound_v(tri_f):
    bound_edge=igl.boundary_facets(tri_f)
    bound_vid=np.unique(bound_edge)
    return bound_vid

In [182]:
bound_vid = get_bound_v(tri_f)

In [183]:
# compute boundary conditions
def compute_bound_cond(tri_v, cage, cage_edge, bound_vid):
    bc = np.zeros((tri_v.shape[0], cage.shape[0]))
    for vi_idx in bound_vid:
        vi = tri_v[vi_idx]
        for v1_idx, v2_idx in cage_edge:
            v1 = cage[v1_idx]
            v2 = cage[v2_idx]
            d1 = np.linalg.norm(vi-v1)
            d2 = np.linalg.norm(vi-v2)
            d12 = np.linalg.norm(v1-v2)
            if -0.0001 < (d12 - (d1 + d2)) / d12 < 0.0001:
                bc[vi_idx,v1_idx] = d2/d12
                bc[vi_idx,v2_idx] = d1/d12
                break
    # check empty bc lines
    assert len(np.where(~bc.any(axis=1))[0]) - (tri_v.shape[0]-bound_vid.shape[0]) == 0
    return bc

In [184]:
bc = compute_bound_cond(tri_v, cage, cage_edge, bound_vid)

#### (2) Solve Lx=0 s.t. boundary conditions

In [185]:
# solve Lx=0 s.t. boundary conditions
def solve_laplacian(tri_v, tri_f, bc):
    l = igl.cotmatrix(tri_v, tri_f)
    m = igl.massmatrix(tri_v, tri_f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.diags(1 / m.diagonal())

    A = l @ minv @ l

    is_bound = np.zeros(tri_v.shape[0])
    is_bound[bound_vid] = 1

    fid = np.argwhere(is_bound==0)[:,0]
    cid = bound_vid

    A_ff = A[fid][:,fid]
    A_fc = A[fid][:,cid]

    xc = bc[cid]

    xf = sp.linalg.spsolve(A_ff, -A_fc @ xc)

    x = np.zeros((tri_v.shape[0], cage.shape[0]))
    x[fid] = xf
    x[cid] = xc

    return x

In [186]:
harmonic_w = solve_laplacian(tri_v, tri_f, bc)

In [187]:
# plot harmonic weight for the first 5 cage vertices
for i in range(5):  
    p = mp.plot(tri_v,tri_f,shading={"wireframe": True,"width":150,"height":150},c=harmonic_w[:,i])
#     p = mp.subplot(tri_v,tri_f,shading={"wireframe": True},c=harmonic_w[:,i],s=[1,5,i])
    p.save("imgs/harmonic_weight%d.html"%i)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4250000…

Plot saved to file imgs/harmonic_weight0.html.


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4250000…

Plot saved to file imgs/harmonic_weight1.html.


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4250000…

Plot saved to file imgs/harmonic_weight2.html.


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4250000…

Plot saved to file imgs/harmonic_weight3.html.


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4250000…

Plot saved to file imgs/harmonic_weight4.html.


### Map object vertices to triangles and compute barycentric coordinates
#### (1) Find the triangle each object vertice lies in.
Point P is inside triangle ABC 
<=> The cross product of PA and PB, PB and PC, PC and PA have the same orientation (same sign, all positive or all negative).

Compute one vertice with all triangles at a time. 

In [188]:
# find the triangle that p is inside
def is_in_triangle(triangles, p):
    P = np.tile(p, triangles.shape[0]).reshape(triangles.shape[0],-1)
    A = triangles[:,0,:]
    B = triangles[:,1,:]
    C = triangles[:,2,:]
    PA = A - P
    PB = B - P
    PC = C - P
    t1 = np.cross(PA, PB)
    t2 = np.cross(PB, PC)
    t3 = np.cross(PC, PA)
    flag1 = np.where(t1 * t2 >= 0, 1, 0)
    flag2 = np.where(t1 * t3 >= 0, 1, 0)
    sign = flag1 * flag2
    pos = np.where(sign==1)
    return pos

In [189]:
# get all triangle positions
triangles = []
for i in range(tri_f.shape[0]):
    triangles.append([tri_v[tri_f[i][0]], tri_v[tri_f[i][1]],tri_v[tri_f[i][2]]])
triangles = np.array(triangles)    


In [190]:
# find triangle id for all vertices
def map_vertix_tri(v,triangles):
    v2tri = []
    for vi in v:
        tri_ids = is_in_triangle(triangles[:,:,:2], np.array(vi[:2]))[0]
        if len(tri_ids)>1:
            print("More than 1 triangle match")
        v2tri.append(tri_ids[0])
    return v2tri

In [191]:
v2tri =  map_vertix_tri(v,triangles)

#### (2) Compute the barycentric coordinate of the vertice within the triangle.

In [192]:
def barycentric_coord(p, triangle):
    a,b,c = triangle
    A = [
        [b[0]-a[0],c[0]-a[0]],
        [b[1]-a[1],c[1]-a[1]]
    ]
    b = [p[0]-a[0], p[1]-a[1]]
    w2, w3 = np.linalg.solve(A,b)
    w1 = 1-w2-w3
    return np.array([w1, w2, w3])

In [193]:
bary_coor = []

for i in range(v.shape[0]):
    vi=np.array(v)[i]
    bary_coor.append(barycentric_coord(vi,triangles[v2tri[i]]))
bary_coor = np.array(bary_coor)

### Compute Harmonic Coordinates

In [194]:
harmonic_coor = np.zeros((v.shape[0], cage.shape[0]))

for i in range(v.shape[0]):
    for j in [0,1,2]:
        harmonic_coor[i,:] += bary_coor[i, j] * harmonic_w[tri_f[v2tri[i],j],:]

## Interactive Deformation

In [205]:
v_copy = v.copy()
tri_v_copy=tri_v.copy()

def position_deformer(tri_v_copy):
    return harmonic_coor @ tri_v_copy[:cage.shape[0],:]

def pos_f(selected_vertices, x,y,z):
    tri_v_copy[selected_vertices,:] = tri_v[selected_vertices,:] + np.array([x,y,z])
    v_deformed = pos_f.deformer(tri_v_copy)

    p.update_object(oid=0, vertices = v_deformed)
    
    global point_oid
    global line_oid
    p.remove_object(point_oid)
    p.remove_object(line_oid)
    point_oid = p.add_points(tri_v_copy[:cage.shape[0]], shading={"point_size":0.15})
    line_oid = p.add_lines(tri_v_copy[cage_edge[:,0]], tri_v_copy[cage_edge[:,1]])
    
pos_f.deformer = position_deformer

In [206]:
def widgets_wrapper():
    select_widget = iw.SelectMultiple(
                            options=np.arange(cage.shape[0]),
                            rows=8,
                            description="Select Cage Vertices")
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0, step=0.01) 
                        for i in 'xyz'}

    widgets_dict = dict(selected_vertices=select_widget)
    widgets_dict.update(translate_widget)
    return widgets_dict

In [207]:
p = mp.plot(v, f,shading={"width":300,"height":300})
point_oid = p.add_points(cage, shading={"point_size":0.15})
line_oid = p.add_lines(cage[cage_edge[:,0]], cage[cage_edge[:,1]])

iw.interact(pos_f, **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

interactive(children=(SelectMultiple(description='Select Cage Vertices', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9…

<function __main__.pos_f>

In [198]:
p.save("imgs/deformation.html")

Plot saved to file imgs/deformation.html.
